In [ ]:
import os
import statistics

# 사용자로부터 폴더 경로 입력받기
folder = "/workspace/seq-vision/UCF101"

# 폴더 내 .avi 파일 목록 생성
avi_files = [f for f in os.listdir(folder) if f.lower().endswith(".avi")]

class_counts = {}

for file in avi_files:
    # 확장자 제거한 파일명
    filename = os.path.splitext(file)[0]
    
    # 두 번째 '_'의 위치 찾기 (첫번째 '_'를 기준으로 그 이후에 다시 찾음)
    first_us = filename.find('_')
    if first_us == -1:
        print(f"파일 '{file}'에는 '_'가 없습니다. 건너뜁니다.")
        continue
    second_us = filename.find('_', first_us + 1)
    if second_us == -1:
        print(f"파일 '{file}'에는 두 번째 '_'가 없습니다. 건너뜁니다.")
        continue

    # 여기서는 파일명의 처음부터 두 번째 '_' 까지의 부분을 클래스 이름으로 사용
    class_label = filename[:second_us]
    
    # 클래스별 카운트 증가
    class_counts[class_label] = class_counts.get(class_label, 0) + 1

if class_counts:
    counts = list(class_counts.values())
    min_count = min(counts)
    mean_count = statistics.mean(counts)
    max_count = max(counts)
    
    print("\n클래스별 파일 개수:")
    for label, cnt in class_counts.items():
        print(f"{label}: {cnt}")
    
    print(f"\n최소 개수: {min_count}")
    print(f"평균 개수: {mean_count:.2f}")
    print(f"최대 개수: {max_count}")
else:
    print("조건에 맞는 .avi 파일이 없거나 모두 건너뛰었습니다.")

In [ ]:
import os
import cv2
import statistics

# 사용자로부터 폴더 경로 입력받기
folder = "/workspace/seq-vision/UCF101"

# 폴더 내 .avi 파일 목록 생성 (파일인지도 확인)
avi_files = [f for f in os.listdir(folder) if f.lower().endswith(".avi") and os.path.isfile(os.path.join(folder, f))]

durations = []  # 각 영상의 재생 시간(초)을 저장할 리스트

for file in avi_files:
    file_path = os.path.join(folder, file)
    cap = cv2.VideoCapture(file_path)
    if not cap.isOpened():
        print(f"파일 '{file}' 열기에 실패하였습니다.")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        print(f"파일 '{file}'의 FPS 값을 읽어오지 못했습니다.")
        cap.release()
        continue

    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps  # 영상 재생 시간(초) 계산
    durations.append(duration)
    cap.release()

if durations:
    min_duration = min(durations)
    mean_duration = statistics.mean(durations)
    max_duration = max(durations)
    
    print("\n전체 .avi 영상들의 재생 시간 통계:")
    print(f"최소 재생 시간: {min_duration:.2f}초")
    print(f"평균 재생 시간: {mean_duration:.2f}초")
    print(f"최대 재생 시간: {max_duration:.2f}초")
else:
    print("재생 시간을 계산할 수 있는 .avi 영상이 없습니다.")

In [ ]:
import os
import cv2
import numpy as np
import random

# --- 1. 파일들을 클래스별로 그룹화 ---
folder = "/workspace/seq-vision/UCF101"

# 폴더 내 .avi 파일 목록
files = [f for f in os.listdir(folder) if f.lower().endswith(".avi") and os.path.isfile(os.path.join(folder, f))]

# 클래스별 파일 경로를 저장할 딕셔너리
class_to_files = {}

for file in files:
    filename = os.path.splitext(file)[0]
    # 첫번째, 두번째 '_'의 위치 찾기
    first_us = filename.find('_')
    if first_us == -1:
        print(f"파일 '{file}'에는 '_'가 없어 건너뜁니다.")
        continue
    second_us = filename.find('_', first_us + 1)
    if second_us == -1:
        print(f"파일 '{file}'에는 두 번째 '_'가 없어 건너뜁니다.")
        continue
    # 클래스 label은 파일명 처음부터 두번째 '_'까지
    class_label = filename[:second_us]
    class_to_files.setdefault(class_label, []).append(os.path.join(folder, file))

# --- 2. 각 클래스별 10개 영상 선택 및 3초 프레임 추출 ---
# 결과를 저장할 딕셔너리 (클래스별 처리 후 분할)
class_to_videos = {}  # 각 영상은 [frame, frame, ...] 리스트로 저장

# 각 영상에 대해 3초 분량의 프레임 추출 (영상을 읽어서 리스트로 저장)
for class_label, file_list in class_to_files.items():
    # 영상이 10개 이상 있는 경우에만 처리 (그렇지 않으면 건너뜁니다)
    if len(file_list) < 10:
        print(f"클래스 '{class_label}'의 영상 개수가 10개 미만({len(file_list)})이어서 건너뜁니다.")
        continue
    # 파일이 10개보다 많으면 무작위로 10개 선택 (순서를 섞어 선택)
    selected_files = random.sample(file_list, 10) if len(file_list) > 10 else file_list
    
    video_list = []  # 해당 클래스의 10개 영상의 프레임 리스트들
    
    for video_path in selected_files:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"영상 '{video_path}' 열기에 실패했습니다.")
            continue
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:
            print(f"영상 '{video_path}'의 FPS를 읽어오지 못했습니다.")
            cap.release()
            continue
        
        # 3초에 해당하는 목표 프레임 개수 (정수)
        target_frames = int(round(fps * 3))
        frames = []
        
        # 3초 동안 프레임 읽기
        while len(frames) < target_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()
        
        # 영상이 3초보다 짧으면 제로 패딩 (첫 프레임의 shape을 기준)
        if len(frames) == 0:
            print(f"영상 '{video_path}'에서 프레임을 하나도 읽지 못했습니다. 건너뜁니다.")
            continue
        if len(frames) < target_frames:
            # 첫 프레임의 shape과 dtype을 가져옴
            frame_shape = frames[0].shape
            dtype = frames[0].dtype
            # 부족한 프레임 수 만큼 제로 패딩
            for _ in range(target_frames - len(frames)):
                frames.append(np.zeros(frame_shape, dtype=dtype))
        
        video_list.append(frames)
    
    if len(video_list) != 10:
        print(f"클래스 '{class_label}'의 영상 처리 결과 10개가 모이지 않았습니다 (모인 개수: {len(video_list)}).")
        # 원하는 경우 부족한 클래스를 건너뛰거나 추가 처리 가능
        continue
    
    class_to_videos[class_label] = video_list

# --- 3. 각 클래스별로 (6, 2, 2) 비율로 분할 ---
train_list = []  # 각 항목: (클래스 라벨, 영상 프레임 리스트)
valid_list = []
test_list = []

for class_label, videos in class_to_videos.items():
    # 10개의 영상 리스트가 있다고 가정하고, 순서를 섞은 후 분할
    random.shuffle(videos)
    train_videos = videos[:6]
    valid_videos = videos[6:8]
    test_videos  = videos[8:10]
    
    # 각 영상에 대해 (클래스 라벨, 영상 프레임 리스트) 형태로 저장
    for vid in train_videos:
        train_list.append((class_label, vid))
    for vid in valid_videos:
        valid_list.append((class_label, vid))
    for vid in test_videos:
        test_list.append((class_label, vid))

# --- 4. 분할 결과 출력 ---
print("\n분할된 결과:")
print(f"훈련 데이터 (train): 총 {len(train_list)}개 영상")
print(f"검증 데이터 (valid): 총 {len(valid_list)}개 영상")
print(f"테스트 데이터 (test): 총 {len(test_list)}개 영상")

# 각 리스트의 첫번째 항목에 대해 간단히 클래스 라벨과 프레임 개수를 출력
def print_sample(dataset, name):
    if dataset:
        sample_label, sample_frames = dataset[0]
        print(f"{name}의 첫 영상: 클래스 = {sample_label}, 프레임 수 = {len(sample_frames)}")
    else:
        print(f"{name} 데이터가 없습니다.")

print_sample(train_list, "훈련")
print_sample(valid_list, "검증")
print_sample(test_list, "테스트")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# -----------------------------
# 1. 전역 label mapping 생성
# -----------------------------
# train_list, valid_list, test_list 는 앞서 구성한 리스트들입니다.
all_labels = set([label for dataset in (train_list, valid_list, test_list) for label, _ in dataset])
global_label2idx = {label: idx for idx, label in enumerate(sorted(all_labels))}

# -----------------------------
# 2. VideoDataset 클래스 정의
# -----------------------------
class VideoDataset(Dataset):
    def __init__(self, video_list, label2idx=None, transform=None):
        """
        video_list: 각 항목이 (클래스 라벨, 영상 프레임 리스트) 형태입니다.
        label2idx: 전역 label mapping을 사용할 수 있습니다.
        transform: 필요시 영상 tensor에 적용할 transform 함수.
        """
        self.video_list = video_list
        self.transform = transform
        if label2idx is None:
            # 만약 mapping이 주어지지 않으면, video_list 내 라벨들을 기준으로 mapping 생성
            self.label2idx = {}
            for label, _ in self.video_list:
                if label not in self.label2idx:
                    self.label2idx[label] = len(self.label2idx)
        else:
            self.label2idx = label2idx

    def __len__(self):
        return len(self.video_list)
    
    def __getitem__(self, idx):
        label, frames = self.video_list[idx]
        # 각 영상은 프레임들의 리스트입니다.
        # 각 프레임은 OpenCV로 읽었을 경우 (H, W, C) 모양의 numpy array 입니다.
        # 이를 torch tensor로 변환하고, (C, H, W)로 순서를 바꿉니다.
        video_tensor = torch.stack([
            torch.from_numpy(frame).permute(2, 0, 1) if frame.ndim == 3 else torch.from_numpy(frame)
            for frame in frames
        ])
        target = self.label2idx[label]
        if self.transform:
            video_tensor = self.transform(video_tensor)
        return video_tensor, target

# -----------------------------
# 3. Dataset 인스턴스 생성
# -----------------------------
train_dataset = VideoDataset(train_list, label2idx=global_label2idx)
valid_dataset = VideoDataset(valid_list, label2idx=global_label2idx)
test_dataset  = VideoDataset(test_list,  label2idx=global_label2idx)

# -----------------------------
# 4. DataLoader 생성
# -----------------------------
batch_size = 4  # 원하는 배치 사이즈로 변경 가능

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# -----------------------------
# 5. 예시: DataLoader에서 배치 하나 확인
# -----------------------------
for videos, labels in train_loader:
    print("Videos batch shape:", videos.shape)  # 예상 shape: (batch_size, num_frames, C, H, W)
    print("Labels:", labels)
    break
